In [1]:
import cppimport.import_hook
import numpy as np

#cppimport.force_rebuild(True)

`cppimport` provides an easy way to compile C++ code. When you execute `import _atlas4py`, this will look for a file `_atlas4py.cpp` and import the file as given in there (check the include paths there!)

In [2]:
import _atlas4py

In [23]:
#grid = _atlas4py.StructuredGrid("L32x32")
#grid = _atlas4py.StructuredGrid(x_spacings=[_atlas4py.LinearSpacing(-1, 1, 21)]*15 + [_atlas4py.LinearSpacing(-1,  1, 15)]*6,
#                                y_spacing=_atlas4py.LinearSpacing(-1, 1, 21))
grid = _atlas4py.StructuredGrid(x_spacing=_atlas4py.LinearSpacing(-1, 1, 20),
                                y_spacing=_atlas4py.LinearSpacing(-1, 1, 21))
print(grid)
mesh = _atlas4py.StructuredMeshGenerator().generate(grid)
_atlas4py.build_edges(mesh)
_atlas4py.build_node_to_edge_connectivity(mesh)
fs = _atlas4py.functionspace.CellColumns(mesh)
print(fs)

_atlas4py.Grid({'domain': {'type': 'rectangular', 'units': 'degrees', 'xmax': 1.0, 'xmin': -1.0, 'ymax': 1.0, 'ymin': -1.0}, 'projection': {'type': 'lonlat'}, 'type': 'structured', 'xspace': {'N': 20, 'end': 1.0, 'endpoint': True, 'start': -1.0, 'type': 'linear'}, 'yspace': {'N': 21, 'end': 1.0, 'endpoint': True, 'start': -1.0, 'type': 'linear'}})


In [4]:
out_f = fs.create_field(name="my_field", levels=1, dtype=np.float64)
in_f = fs.create_field(name="my_field", levels=1, dtype=np.float64)

You can create views on the atlas fields. Don't forget to pass `copy=False`! The views can be accessed like normal numpy arrays

In [5]:
out_view = np.array(out_f, copy=False)
in_view = np.array(in_f, copy=False)
out_view[:] = in_view[:] = np.zeros_like(in_view)

In [6]:
lonlat_view = np.array(mesh.nodes.lonlat, copy=False)
lonlat0 = lonlat_view[0]
lonlat1 = lonlat_view[-1]

In [7]:
assert in_view.shape[1] == 1
for cPos in range(mesh.cells.size):
    nPos = mesh.cells.node_connectivity[cPos, 0]
    center_lon = (lonlat_view[nPos, 0] - lonlat0[0] + (lonlat0[0] - lonlat1[0]) / 2)
    center_lat = (lonlat_view[nPos, 1] - lonlat0[1] + (lonlat0[1] - lonlat1[1]) / 2)
    in_view[cPos, 0] = 1 if abs(center_lon) < .5 and abs(center_lat) < .5 else 0
    out_view[cPos, 0] = 0

Now we also compile and import `computation.cpp`. Note that there is an absolute path to `gmsh` in here.

In [8]:
!rm out.msh
import computation
with _atlas4py.Gmsh("out.msh") as out:
    out.write(mesh)
    for i in range(100):
        in_f.metadata["step"] = i
        out.write(in_f)
        computation.run_computation(mesh, 1, in_f, out_f)
        in_f, out_f = out_f, in_f
!~/packages/gmsh/gmsh-4.4.1-Linux64/bin/gmsh out.msh 

rm: cannot remove 'out.msh': No such file or directory


# Learning about atlas...

In [9]:
grid = _atlas4py.StructuredGrid(x_spacing=_atlas4py.LinearSpacing(-1, 1, 5),
                                y_spacing=_atlas4py.LinearSpacing(-1, 1, 5))
mesh = _atlas4py.StructuredMeshGenerator().generate(grid)
_atlas4py.build_edges(mesh)
_atlas4py.build_node_to_edge_connectivity(mesh)
fs = _atlas4py.functionspace.CellColumns(mesh, halo=2)

In [10]:
mesh.cells.size

16

In [11]:
mesh.edges.size

40

In [12]:
mesh.nodes.size

25

In [13]:
lonlat_view = np.array(mesh.nodes.lonlat, copy=False)
lonlat_view

array([[-1. , -1. ],
       [-0.5, -1. ],
       [ 0. , -1. ],
       [ 0.5, -1. ],
       [ 1. , -1. ],
       [-1. , -0.5],
       [-0.5, -0.5],
       [ 0. , -0.5],
       [ 0.5, -0.5],
       [ 1. , -0.5],
       [-1. ,  0. ],
       [-0.5,  0. ],
       [ 0. ,  0. ],
       [ 0.5,  0. ],
       [ 1. ,  0. ],
       [-1. ,  0.5],
       [-0.5,  0.5],
       [ 0. ,  0.5],
       [ 0.5,  0.5],
       [ 1. ,  0.5],
       [-1. ,  1. ],
       [-0.5,  1. ],
       [ 0. ,  1. ],
       [ 0.5,  1. ],
       [ 1. ,  1. ]])

In [14]:
import pprint
print("number of blocks: ", mesh.edges.node_connectivity.blocks)

for block in range(mesh.edges.node_connectivity.blocks):
    print("\nblock: ", block)
    b = mesh.edges.node_connectivity.block(block)
    print("{} x {}".format(b.rows, b.cols))
    pprint.pprint([[b[i, j] for j in range(b.cols)] for i in range(b.rows)])
    print("--- ")


number of blocks:  1

block:  0
40 x 2
[[5, 0],
 [5, 6],
 [6, 1],
 [0, 1],
 [6, 7],
 [7, 2],
 [1, 2],
 [7, 8],
 [8, 3],
 [2, 3],
 [8, 9],
 [9, 4],
 [3, 4],
 [10, 5],
 [10, 11],
 [11, 6],
 [11, 12],
 [12, 7],
 [12, 13],
 [13, 8],
 [13, 14],
 [14, 9],
 [15, 10],
 [15, 16],
 [16, 11],
 [16, 17],
 [17, 12],
 [17, 18],
 [18, 13],
 [18, 19],
 [19, 14],
 [20, 15],
 [20, 21],
 [21, 16],
 [21, 22],
 [22, 17],
 [22, 23],
 [23, 18],
 [23, 24],
 [24, 19]]
--- 


The number of blocks are 2. If we build pole edges, these edges get into the second block.

In [15]:
print("number of blocks: ", mesh.cells.node_connectivity.blocks)

for block in (0, 1):
    print("\nblock: ", block)
    b = mesh.cells.node_connectivity.block(block)
    print("{} x {}".format(b.rows, b.cols))
    pprint.pprint([[b[i, j] for j in range(b.cols)] for i in range(b.rows)])
    print("--- ")


number of blocks:  2

block:  0
16 x 4
[[0, 5, 6, 1],
 [1, 6, 7, 2],
 [2, 7, 8, 3],
 [3, 8, 9, 4],
 [5, 10, 11, 6],
 [6, 11, 12, 7],
 [7, 12, 13, 8],
 [8, 13, 14, 9],
 [10, 15, 16, 11],
 [11, 16, 17, 12],
 [12, 17, 18, 13],
 [13, 18, 19, 14],
 [15, 20, 21, 16],
 [16, 21, 22, 17],
 [17, 22, 23, 18],
 [18, 23, 24, 19]]
--- 

block:  1
0 x 3
[]
--- 


In [16]:
print("number of blocks: ", mesh.cells.edge_connectivity.blocks)

for block in range(mesh.cells.edge_connectivity.blocks):
    print("\nblock: ", block)
    b = mesh.cells.edge_connectivity.block(block)
    print("{} x {}".format(b.rows, b.cols))
    pprint.pprint([[b[i, j] for j in range(b.cols)] for i in range(b.rows)])
    print("--- ")


number of blocks:  2

block:  0
16 x 4
[[1, 0, 2, 3],
 [4, 2, 5, 6],
 [7, 5, 8, 9],
 [10, 8, 11, 12],
 [14, 13, 15, 1],
 [16, 15, 17, 4],
 [18, 17, 19, 7],
 [20, 19, 21, 10],
 [23, 22, 24, 14],
 [25, 24, 26, 16],
 [27, 26, 28, 18],
 [29, 28, 30, 20],
 [32, 31, 33, 23],
 [34, 33, 35, 25],
 [36, 35, 37, 27],
 [38, 37, 39, 29]]
--- 

block:  1
0 x 3
[]
--- 


In [17]:
c = mesh.nodes.edge_connectivity
pprint.pprint([[c[i, j] for j in range(c.cols(i))] for i in range(c.rows)])


[[0, 3],
 [2, 3, 6],
 [5, 6, 9],
 [8, 9, 12],
 [11, 12],
 [13, 1, 0],
 [15, 1, 4, 2],
 [17, 4, 7, 5],
 [19, 7, 10, 8],
 [21, 10, 11],
 [22, 14, 13],
 [24, 14, 16, 15],
 [26, 16, 18, 17],
 [28, 18, 20, 19],
 [30, 20, 21],
 [31, 23, 22],
 [33, 23, 25, 24],
 [35, 25, 27, 26],
 [37, 27, 29, 28],
 [39, 29, 30],
 [32, 31],
 [32, 34, 33],
 [34, 36, 35],
 [36, 38, 37],
 [38, 39]]


Test whether you can write different types into the metadata and get back the correct type.

In [21]:
print(in_f.metadata)

in_f.metadata["bool"] = True
print(in_f.metadata["bool"])

in_f.metadata["int"] = 3
print(in_f.metadata["int"])

in_f.metadata["float"] = 3.12
print(in_f.metadata["float"])

in_f.metadata["str"] = "x"
print(in_f.metadata["str"])

print(in_f.metadata)

_atlas4py.Metadata({'bool': True, 'float': 3.12, 'global': False, 'int': 3, 'levels': 1, 'name': 'my_field', 'step': 98, 'str': 'x', 'variables': 0})
True
3
3.12
x
_atlas4py.Metadata({'bool': True, 'float': 3.12, 'global': False, 'int': 3, 'levels': 1, 'name': 'my_field', 'step': 98, 'str': 'x', 'variables': 0})
